In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms as transforms
from torchvision import datasets
import matplotlib.pyplot as plt
import numpy as np
from ipynb.fs.full.AI_Image_Data_Class import AI_Image_Data
import time
import os

In [6]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


## Old way to load data using AI_Image_Data class


In [ ]:
training_data = AI_Image_Data("train.csv", transform=transform)
testing_data = AI_Image_Data("small_test.csv", transform=transform)

In [13]:
print(len(testing_data))

100


In [ ]:
train_loader = torch.utils.data.DataLoader(training_data, batch_size=4,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(testing_data, batch_size=1,
                                          shuffle=True)

In [ ]:
dataiter = iter(train_loader)
images, labels = next(dataiter)

## New Way using ImageFolder

In [13]:
data_dir = "Images"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform)
                  for x in ['train', 'test']}

100000


In [14]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes
print(class_names)

['FAKE', 'REAL']


In [15]:
inputs, classes = next(iter(dataloaders['train']))

In [16]:
print(len(dataloaders['train']))

3125


## Defining ConvNet

In [17]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
        
    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
num_epochs = 5
learning_rate = 0.01

In [14]:
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training

In [16]:
n_total_steps = len(train_loader)
num_epochs = 5
start_time = time.time()
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 1000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
print("--- %s seconds ---" % (time.time() - start_time))

Epoch [1/5], Step [1000/25000], Loss: 0.7059
Epoch [1/5], Step [2000/25000], Loss: 0.6466
Epoch [1/5], Step [3000/25000], Loss: 0.6439
Epoch [1/5], Step [4000/25000], Loss: 0.6349
Epoch [1/5], Step [5000/25000], Loss: 0.6896
Epoch [1/5], Step [6000/25000], Loss: 0.6377
Epoch [1/5], Step [7000/25000], Loss: 0.7273
Epoch [1/5], Step [8000/25000], Loss: 0.6897
Epoch [1/5], Step [9000/25000], Loss: 0.5322
Epoch [1/5], Step [10000/25000], Loss: 0.3965
Epoch [1/5], Step [11000/25000], Loss: 0.4670
Epoch [1/5], Step [12000/25000], Loss: 0.4474
Epoch [1/5], Step [13000/25000], Loss: 0.4276
Epoch [1/5], Step [14000/25000], Loss: 0.7679
Epoch [1/5], Step [15000/25000], Loss: 0.5353
Epoch [1/5], Step [16000/25000], Loss: 0.7118
Epoch [1/5], Step [17000/25000], Loss: 0.5007
Epoch [1/5], Step [18000/25000], Loss: 0.3586
Epoch [1/5], Step [19000/25000], Loss: 0.3785
Epoch [1/5], Step [20000/25000], Loss: 0.4216
Epoch [1/5], Step [21000/25000], Loss: 0.5788
Epoch [1/5], Step [22000/25000], Loss: 0.73

### Saving Model

In [17]:
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

### Loading Model

In [7]:
loaded_model = ConvNet().to(device)
loaded_model.load_state_dict(torch.load("cnn.pth"))

<All keys matched successfully>

In [8]:
loaded_model.eval()

OrderedDict([('conv1.weight', tensor([[[[ 9.6011e-02,  6.3040e-02, -5.4826e-02,  1.3691e-01,  2.2186e-01],
          [ 1.1329e-01,  6.6459e-02, -1.4016e-01, -7.4858e-03,  1.3445e-01],
          [-2.0845e-02, -8.8221e-03, -1.8006e-01,  5.0967e-02,  9.2051e-02],
          [ 1.5262e-02,  4.8001e-02, -1.3303e-02,  8.2560e-02,  4.7212e-02],
          [ 1.8711e-01,  1.1377e-01, -1.4667e-02,  2.0488e-01,  1.8961e-02]],

         [[-5.6807e-02,  2.7736e-03, -7.7165e-02, -9.9446e-02,  1.2723e-01],
          [ 7.1058e-02,  4.4817e-02, -2.1640e-01, -4.8209e-02,  1.3640e-01],
          [ 3.5053e-02, -7.9112e-03, -1.8735e-01,  3.6261e-03,  7.2376e-02],
          [-1.2039e-02, -6.3834e-02, -5.0310e-02,  8.6946e-02,  8.1362e-03],
          [ 1.2091e-01,  2.2088e-01,  8.8482e-02,  9.9542e-02,  4.8358e-02]],

         [[ 7.8724e-02,  9.9828e-02, -9.1687e-02, -2.8867e-02,  2.3321e-01],
          [ 4.2321e-02,  1.0339e-01, -1.5290e-01,  6.3837e-02,  1.2818e-01],
          [-3.1588e-02, -1.4434e-01, -7.64

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')


In [15]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = loaded_model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the {len(test_loader)} test images: {acc} %')

Accuracy of the network on the 100 test images: 87.0 %
